# Interacting with GPIO from MicroBlaze

In [34]:
from pynq.overlays.base import BaseOverlay
import time
import threading
import multiprocessing
import os
import socket
from multiprocessing import Process, Value, Array

base = BaseOverlay("base.bit")

In [35]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
void write_gpioB(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpioB(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Add a C++ function to reset all the GPIO pins on the chosen PMOD
void reset_gpioB(){
    for (unsigned int j = 0; j <= 7; j = j + 1) { 
        write_gpioB(j, 0);
        //write_gpio(pin,val)
        //we want the value reset to zero
        //12 pins, starts at 0
    }
}

In [36]:
#LED_Status = [0, 0, 2, 3] #Default Status
#LED_Status = [1, 1, 2, 1] #test
start_LED = False #when connection is made, LED() will start
running = True #when client breaks connection, LED() sill stop running

sleep_time = 1
frequency = 15 #for the PWM Brightness
Duty_Cycle = [0.04736614270344993, 0.217637640824031, 0.5310492251033824, 1.0] #25%, 50%, 75%, and 100% brightness.
#time_on = duty_cycle * (1/frequency) && time_off = (1-duty_cycle) * (1/frequency)
PWM_on = []
for i in range(4):
    PWM_on.append(Duty_Cycle[i]/frequency)
PWM_off = []
for i in range(4):
    PWM_off.append((1-Duty_Cycle[i])/frequency)
PWM_run_time = frequency*sleep_time

In [37]:
def LED():
    global LED_Status, PWM_on, PWM_off, PWM_run_time, sleep_time, running
    while running:
        if LED_Status[0] == 0: #IF OFF
            time.sleep(sleep_time)
        else: #IF ON
            if LED_Status[3] == 3: #full bright (NO PWM)
                if LED_Status[2] == 0: #If NO PWM & Cycle Color
                    for i in range(1,4): #cycle write from 1 (blue), 2 (red) and 3 (green)
                        write_gpioB(i,1) #turn on  LED
                        time.sleep(sleep_time) #sleep with ON
                        write_gpioB(i,0) #turn off LED
                        if LED_Status[1] == 1: #if supposed to blink, sleep
                            time.sleep(sleep_time)
                else: #IF NO PWM & Solid Color
                    write_gpioB(LED_Status[2],1)
                    time.sleep(sleep_time)
                    if LED_Status[1] == 1: #if supposed to blink, sleep
                            write_gpioB(LED_Status[2],0)
                            time.sleep(sleep_time)
            else: #If NOT full brightness (PWM needed)
                if LED_Status[2] == 0: #PWM & cycle thru colors
                    for i in range(1,4):
                        for j in range(PWM_run_time):
                            write_gpioB(i,1) #turn on LED
                            time.sleep(PWM_on[LED_Status[3]]) #sleep with ON
                            write_gpioB(i,0) #turn off LED
                            time.sleep(PWM_off[LED_Status[3]]) #sleep with OFF
                        if LED_Status[1] == 1: #If blinking
                            time.sleep(sleep_time)
                else:
                    for i in range(PWM_run_time):
                        write_gpioB(LED_Status[2],1) #turn on LED
                        time.sleep(PWM_on[LED_Status[3]]) #sleep with ON
                        write_gpioB(LED_Status[2],0) #turn off LED
                        time.sleep(PWM_off[LED_Status[3]]) #sleep with OFF
                    if LED_Status[1] == 1: #If Blinking
                        time.sleep(sleep_time)

def Wait_LED(start):
    global sleep_time
    print('Waiting for LED to start.')
    while True:
        if start.value == 1:
            print('Starting LED')
            LED()
            break
        print('waiting... ', start)
        time.sleep(sleep_time)
        
def test(start):
    itera = 0
    while True:
        itera +=1
        if itera>5:
            start.value = 1
            print('start_LED = ',start.value)
            #print(start_LED)
            break
        print(itera)
        time.sleep(1)

In [ ]:
def run_pynq_server_R():
    global LED_Status, start_LED
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('192.168.2.99', 12345))
    sock.listen()
    conn, addr = sock.accept()
    print("Ramin's Server is waiting for a connection!")
    with conn:
        print("{} connected to Ramin's Server!".format(addr))
        start_LED = True #Now we can allow the LED() function to begin.
        while True:
            time.sleep(0.0001)
            data = conn.recv(1024)
            data = data.decode()
            if (data == 'disconnect'):
                print("Client is disconnecting from Ramin's Server.")
                running = False
                break
            data = list(data) #Here we convert from string into list, then update all indexes from str to int
            for i in range(4): #length of data list ALWAYS = 4, could do 'len(data)' instead though.
                LED_Status[i] = int(data[i])
    print("Client has disconnected from Ramin's Server.")


In [ ]:
p_server = multiprocessing.Process(target=run_pynq_server_R)
p_server.start()
os.system("taskset -p -c {} {}".format(0, p_server.pid))

p_GPIOB = multiprocessing.Process(target=Wait_LED)
p_GPIOB.start()
os.system("taskset -p -c {} {}".format(1, p_GPIOB.pid))

p_server.join();
p_GPIOB.join();

In [38]:
reset_gpioB()

start = Value('i', 0)
LED_Status = Array('i', [1, 0, 2, 2])

p_GPIOB = multiprocessing.Process(target=test,args=(start,LED_Status))
p_GPIOB.start()
os.system("taskset -p -c {} {}".format(1, p_GPIOB.pid))
p_server = multiprocessing.Process(target=Wait_LED,args=(start,LED_Status))
p_server.start()
os.system("taskset -p -c {} {}".format(0, p_server.pid))

1
Waiting for LED to start.
waiting...  <Synchronized wrapper for c_long(0)>


0

2
waiting...  <Synchronized wrapper for c_long(0)>
3
waiting...  <Synchronized wrapper for c_long(0)>
4
waiting...  <Synchronized wrapper for c_long(0)>
5
waiting...  <Synchronized wrapper for c_long(0)>
start_LED =  1
Starting LED


Process Process-20:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-37-1a096acd61cd>", line 46, in Wait_LED
    LED()
  File "<ipython-input-37-1a096acd61cd>", line 36, in LED
    time.sleep(PWM_off[LED_Status[3]]) #sleep with OFF
KeyboardInterrupt
